In [ ]:
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from custom_functions import processing
import pandas as pd

In [ ]:
dataset = pd.read_excel('data_v10.xlsx')
dataset = dataset.drop(['Foldername'], axis=1)
dataset.head()

,Gender,VINCQ32DDN,VINICODEX003,FROPCOM0001,FROPCOM0005,FROPCOM0006_S1_,FROPCOM0006_S2_,FROPCOM0006_S3_,FROPCOM0006_S4_,FROPCOM0006_S5_,...,HADS_D_Score,grip,walk_time_4m,Item_1,Item_2,Item_3,Item_4,Item_5,Fried_Score,Fried_State
0,0,76.0,0.0,0.0,3.0,NaN,NaN,NaN,NaN,1.0,...,9,NaN,8.45,0.0,0,1.0,1.0,NaN,2,0
1,0,75.0,1.0,2.0,3.0,0.0,0.0,0.0,0.0,0.0,...,9,21.5,6.11,0.0,0,1.0,1.0,1.0,3,1
2,0,67.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,10,23.2,20.00,0.0,0,0.0,1.0,1.0,2,0
3,0,72.0,1.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,...,15,17.7,4.87,0.0,1,0.0,0.0,1.0,2,0
4,1,69.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10,40.0,3.48,0.0,0,0.0,0.0,1.0,1,0


In [ ]:
# feature names
feature_names = dataset.drop(['Fried_State','Fried_Score','Frailty_State','Frailty_Score',
                              'Item_1', 'Item_2', 'Item_3', 'Item_4', 'Item_5',
                              'Weight_Diff', 'HADS_D_Score', 'walk_time_4m', 'EXAMCLIN02', 'grip'], axis=1).columns.to_list()

# Features and Targets
X = dataset.drop(['Fried_State','Fried_Score','Frailty_State','Frailty_Score',
                  'Item_1','Item_2','Item_3','Item_4','Item_5',
                  'Weight_Diff', 'HADS_D_Score','walk_time_4m','EXAMCLIN02','grip'], axis=1).values

# The two tasks are Fried_State and Frailty_State
y = dataset[['Fried_State','Frailty_State']].values # The two tasks are 'Fried_State' and 'Frailty_State'


In [ ]:
# Imputation
X_imputed, _ = processing(X, 5, [1,3,5,7,9], False)
X_imputed.shape

(97, 807)

In [ ]:
# Define the objective function with Focal Loss and a regularization term for feature sharing
def objective_function_focal(params, X, y, lambda_reg=0.1, alpha=0.25, gamma=2.0):
    num_features = X.shape[1]
    num_tasks = y.shape[1]

    # Reshape params into W (common feature matrix)
    W = params.reshape((num_features, num_tasks))

    # Compute the predictions (logits)
    logits = X.dot(W)

    # Clip logits for numerical stability       [TO DIG]
    clipped_logits = np.clip(logits, -500, 500)

    # Compute probabilities using sigmoid function
    probs = 1 / (1 + np.exp(-clipped_logits))
    #probs = 1 / (1 + np.exp(-logits))

    # Clip probabilities for numerical stability    [TO DIG]
    probs = np.clip(probs, 1e-15, 1 - 1e-15)

    # Compute the Focal Loss for each task
    loss = -np.sum(alpha * ((1 - probs) ** gamma) * y * np.log(probs) + (1 - alpha) * (probs ** gamma) * (1 - y) * np.log(1 - probs))

    # Add regularization term (Frobenius norm)
    loss += lambda_reg * np.linalg.norm(W, 'fro')

    return loss



In [ ]:
# Define the objective function with Hinge Loss and a regularization term for feature sharing
def objective_function_hinge(params, X, y, lambda_reg=0.1):
    num_features = X.shape[1]
    num_tasks = y.shape[1]

    # Reshape params into W (common feature matrix)
    W = params.reshape((num_features, num_tasks))

    # Compute the predictions (logits)
    logits = X.dot(W)

    # Convert labels to +1 and -1 for hinge loss
    y_hinge = 2 * y - 1

    # Compute the Hinge Loss for each task
    loss = np.sum(np.maximum(0, 1 - y_hinge * logits))

    # Add regularization term (Frobenius norm)
    loss += lambda_reg * np.linalg.norm(W, 'fro')

    return loss


In [ ]:
# Initial guess for the common feature matrix W
initial_guess = np.random.rand(X_imputed.shape[1] * y.shape[1])

# Run the optimization to minimize the objective function with Focal Loss
result_focal = minimize(fun=objective_function_focal,
                        x0=initial_guess,
                        args=(X_imputed, y, 0.1, 0.5, 5.0),
                        method='L-BFGS-B')


In [ ]:
# intitial guess for the common feature matrix W
initial_guess2 = np.random.rand(X_imputed.shape[1] * y.shape[1])

# Run the optimization to minimize the objective function with Hinge Loss
result_hinge = minimize(
    fun=objective_function_hinge,
    x0=initial_guess2,
    args=(X_imputed, y, 0.2),  # lambda_reg set to 0.1
    method='L-BFGS-B'
)

In [ ]:
# Extract the optimized W (common feature matrix) for Focal Loss
optimized_W_focal = result_focal.x.reshape((X_imputed.shape[1], y.shape[1]))
optimized_W_focal


In [ ]:
# Extract the optimized W (common feature matrix) for Hinge Loss
optimized_W_hinge = result_hinge.x.reshape((X_imputed.shape[1], y.shape[1]))
optimized_W_hinge


In [ ]:
# Create a mapping
feature_to_weight = {}
for i, feature in enumerate(feature_names):
    feature_to_weight[feature] = optimized_W_hinge[i, :]


## rMTFL

In [ ]:
import numpy as np
from scipy.optimize import minimize

# Custom function placeholderscor
def FGLasso_projection(W, lambda_):
    # solve it in row wise (L_{2,1} is row coupled).
    # for each row we need to solve the proximal opterator
    # argmin_w { 0.5 \|w - v\|_2^2 + lambda_3 * \|w\|_2 }
    Wp = np.multiply(np.maximum(0, 1 - lambda_/np.sqrt(np.sum(np.square(W), axis=1))), W)
    return Wp

def unit_grad_eval(W, C, X, y):
    m = len(y)
    weight = np.ones(m) / m
    weighty = weight * y

    aa = -y * (np.dot(X, W) + C)
    bb = np.maximum(aa, 0)

    funcVal = np.dot(weight, np.log(np.exp(-bb) + np.exp(aa - bb)) + bb)

    pp = 1 / (1 + np.exp(aa))

    b = -weighty * (1 - pp)
    grad_c = np.sum(b)
    grad_w = np.dot(X, b)
    return grad_w, grad_c, funcVal

def gradVal_eval(W, C, X, y):
    dimension, task_num = W.shape
    grad_W = np.zeros((dimension, task_num))
    grad_C = np.zeros(task_num)
    lossValVect = np.zeros(task_num)

    for i in range(task_num):
        grad_W[:, i], grad_C[i], lossValVect[i] = unit_grad_eval(W[:, i], C[i], X[i], y[i])

    funcVal = np.sum(lossValVect)

    return grad_W, grad_C, funcVal


def unit_funcVal_eval(W, C, X, y):
    m = len(y)
    weight = np.ones(m) / m
    aa = -y * (np.dot(X.T, W) + C)
    bb = np.maximum(aa, 0)
    funcVal = np.dot(weight, (np.log(np.exp(-bb) + np.exp(aa - bb)) + bb))
    return funcVal

def funVal_eval(W, C, X, y):
    task_num = len(C)
    funcVal = np.sum([unit_funcVal_eval(W[:, i], C[i], X[i], y[i]) for i in range(task_num)])
    return funcVal


def nonsmooth_eval(W, rho):
    non_smooth_value = np.sum(np.sqrt(np.sum(np.square(W), axis=1))) * rho
    return non_smooth_value

# Initialize parameters
task_num = 2
dimension = 807  # Assuming 807 features
rho1 = 0.1
rho2 = 0.1
maxIter = 100  # Maximum number of iterations
gamma_inc = 2  # Gamma increment in line search

# Initialize P, Q, C (use your actual data)
P0 = np.random.randn(dimension, task_num)
Q0 = np.random.randn(dimension, task_num)
C0 = np.zeros(task_num)

# Placeholder for your data (use your actual data)
X = [np.random.randn(100, dimension) for _ in range(task_num)]
Y = [np.random.choice([-1, 1], size=100) for _ in range(task_num)]

# Initialize variables for the main loop
Pz = P0
Qz = Q0
Cz = C0
Pz_old = P0
Qz_old = Q0
Cz_old = C0
t = 1
t_old = 0
iter = 0
gamma = 1
L = []
F = []

while iter < maxIter:
    alpha = (t_old - 1) / t
    Ps = (1 + alpha) * Pz - alpha * Pz_old
    Qs = (1 + alpha) * Qz - alpha * Qz_old
    Cs = (1 + alpha) * Cz - alpha * Cz_old

    # Compute function value and gradients at the search point
    gWs, gCs, Fs = gradVal_eval(Ps + Qs, Cs, X, y)

    # Armijo-Goldstein line search
    while True:
        Pzp = FGLasso_projection(Ps - gWs/gamma, rho1 / gamma)
        Qzp = FGLasso_projection(Qs.T - gWs.T/gamma, rho2 / gamma).T
        Czp = Cs - gCs/gamma
        Fzp = funVal_eval(Pzp+Qzp, Czp)

        delta_Pzp = Pzp - Ps
        delta_Qzp = Qzp - Qs
        delta_Czp = Czp - Cs

        nm_delta_Pzp = np.linalg.norm(delta_Pzp, 'fro')**2
        nm_delta_Qzp = np.linalg.norm(delta_Qzp, 'fro')**2
        nm_delta_Czp = np.linalg.norm(delta_Czp, 'fro')**2
        r_sum = (nm_delta_Pzp+nm_delta_Qzp+nm_delta_Czp)/2

        Fzp_gamma = Fs + np.sum(np.sum((delta_Pzp + delta_Qzp) * gWs)) \
                    + np.sum(np.sum(delta_Czp * gCs)) \
                    + gamma / 2 * (nm_delta_Pzp + nm_delta_Qzp) \
                    + gamma / 2 * nm_delta_Czp

        if (r_sum <=1e-20):
            break
        if (Fzp <= Fzp_gamma):
            break
        else:
            gamma = gamma * gamma_inc

    Pz_old = Pz
    Qz_old = Qz
    Cz_old = Cz
    Pz = Pzp
    Qz = Qzp
    Cz = Czp
    L = np.concatenate((L, Fzp + nonsmooth_eval(Pzp, rho1) + nonsmooth_eval(Qzp.T, rho2)))
    F = np.concatenate((F, Fzp))

    # Check convergence criteria here
    # If met, break
    # test stop condition.
    if opts['tFlag'] == 0:
        if iter >= 2:
            if abs(L[-1] - L[-2]) <= opts['tol']:
                break
    elif opts['tFlag'] == 1:
        if iter >= 2:
            if abs(L[-1] - L[-2]) <= opts['tol'] * L[-2]:
                break
    elif opts['tFlag'] == 2:
        if L[-1] <= opts['tol']:
            break
    elif opts['tFlag'] == 3:
        if iter >= opts['maxIter']:
            break
iter = iter + 1
t_old = t
t = 0.5 * (1 + (1 + 4 * t**2)**0.5)


# Final optimized P, Q, C are in Pz, Qz, Cz
